# Information Retrieval Project
## COVID-19 Search Engine (P12)

Content-based image retrieval (CBIR) is a computer vision technique which addresses the problem of searching for digital images in large databases. A content-based approach exploits the contents of an image, such as colors, shapes and textures, differing from its concept-based counterpart, which instead focuses on keywords and tags associated with the image itself.

Image retrieval has gained more and more relevance in the medical field, due to the accumulation of extensive collections of scans in hospitals. These images are stored in DICOM format, which must be manually annotated and may require considerable time to process by physicians. The goal of this project is trying to address this problem by considering different approaches for building a content-based medical image retrieval system and comparing their results based on classification metrics and computational time.

## Install the requirements

In [ ]:
!pip install -r requirements.txt

# Define all the import

In [22]:
import os
import kaggle
import platform

## Version Control

In [23]:
# Checking Python version
print("- Python version: {}".format(platform.python_version()))

- Python version: 3.12.0


## Define Useful Constant

In [24]:
# PATHS AND SIMILAR
DATASET_PATH = "archives"  # Path to dataset folder
DATASET_ID = "plameneduardo/sarscov2-ctscan-dataset"  # Necessary for download dataset (Kaggle API)

## Download the dataset from Kaggle

In [25]:
# Download the dataset if not exist in the workplace
if not os.path.exists(DATASET_PATH):
    print("\n> Download the dataset from Kaggle...")
    # Download dataset and unzip it
    kaggle.api.dataset_download_files(dataset=DATASET_ID, path=DATASET_PATH, quiet=False, unzip=True)
else:
    print("\n> Dataset alredy downloaded.")


> Dataset alredy downloaded.
